In [41]:
import pymysql
import pymysql.cursors
import pandas as pd

my_creds = []
with open('../../../staff/admin_mysql_credentials.txt', 'r') as f:
    for line in f.readlines():
        my_creds.append(line.strip())
        
connection = pymysql.connect(host='mysql',
                             user=my_creds[0],
                             database='PHYS7332',
                             password=my_creds[1],
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [18]:
# creating FL bay species table
with connection.cursor() as cursor:
    sql = """
    CREATE TABLE `fla_species` (
        `node_id` int(11) NOT NULL,
        `name` varchar(255)  COLLATE utf8mb4_bin NOT NULL,
        `my_group` varchar(255) COLLATE utf8mb4_bin NULL,
    PRIMARY KEY (`node_id`))
    ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;
    """
    cursor.execute(sql)
connection.commit()

In [19]:
# populating FL bay species
df1 = pd.read_csv('./data/Florida-bay-meta.csv')
insert_sql = """
INSERT INTO `fla_species` (node_id, name, my_group) 
values (%s, %s, %s);
"""
for _, row in df1.iterrows():
    sp_name = row['name']
    node_id = row['node_id']
    if row['group'] == row['group']:
        group = row['group']
    else:
        group = 'NULL'
    with connection.cursor() as cursor:
        cursor.execute(insert_sql, (node_id, sp_name, group))
connection.commit()

In [37]:
# creating food web table
with connection.cursor() as cursor:
    # sql = """DROP TABLE `fla_foodweb`;"""
    # cursor.execute(sql)
    sql = """
    CREATE TABLE `fla_foodweb` (
        `RowId` int(11) NOT NULL,
        `FromNodeId` int(11) NOT NULL,
        `ToNodeId` int(11)  NOT NULL,
    PRIMARY KEY (`RowId`))
    ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_bin;
    """
    cursor.execute(sql)
connection.commit()

In [38]:
# populating food web table
df2 = pd.read_csv('data/Florida-bay.txt', sep=' ', header=4)
df2.columns = ['FromNodeId', 'ToNodeId', 'Extra']
insert_sql = """
INSERT INTO `fla_foodweb` (RowId, FromNodeId, ToNodeId) 
values (%s, %s, %s);
"""
for idx, row in df2.iterrows():
    row_id = idx
    from_node_id = row['FromNodeId']
    to_node_id = row['ToNodeId']
    with connection.cursor() as cursor:
        cursor.execute(insert_sql, (row_id, from_node_id, to_node_id))
connection.commit()

In [45]:
df_body = pd.read_csv('../data_not_in_git/soc-redditHyperlinks-body.tsv', sep='\t')
df_head = pd.read_csv('../data_not_in_git/soc-redditHyperlinks-title.tsv', sep='\t')

In [65]:
from sqlalchemy import create_engine

my_creds = []
with open('../../../staff/admin_mysql_credentials.txt', 'r') as f:
    for line in f.readlines():
        my_creds.append(line.strip())

hostname="mysql-0005"
dbname="PHYS7332"
uname=my_creds[0]
pwd=my_creds[1]

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                           .format(host=hostname, db=dbname,
                                   user=uname,pw=pwd),
                      connect_args={'connect_timeout': 100},
                       pool_pre_ping=True)

In [66]:
df_body['link_source'] = 'body'
df_head['link_source'] = 'title'
df_both = pd.concat([df_body, df_head])

In [69]:
with engine.connect() as conn:
    ix = 0
    while ix < len(df_both):
        tran = conn.begin()
        try:
            df_both[[
                'SOURCE_SUBREDDIT', 
                'TARGET_SUBREDDIT', 
                'POST_ID', 
                'TIMESTAMP', 
                'LINK_SENTIMENT', 
                'link_source']
            ].iloc[ix: ix + 100].to_sql('reddit_hyperlinks', conn, if_exists='append')
            tran.commit()
            ix += 100
        except Exception as e:
            print(e)
            tran.rollback()
    